In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('C:/Users/Administrator/Downloads/research_articles.csv')

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
from gensim import corpora,models

In [6]:
from nltk.tokenize import RegexpTokenizer

In [7]:
from nltk.corpus import stopwords

In [9]:
from nltk.stem import WordNetLemmatizer

In [10]:
df.head()

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8989 entries, 0 to 8988
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        8989 non-null   int64 
 1   TITLE     8989 non-null   object
 2   ABSTRACT  8989 non-null   object
dtypes: int64(1), object(2)
memory usage: 210.8+ KB


In [13]:
df.shape

(8989, 3)

In [15]:
tokenizer=RegexpTokenizer(r'\w+')

In [22]:
lemmatizer=WordNetLemmatizer()
import nltk

In [31]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...


In [32]:
def clean_text(text):
    tokens=tokenizer.tokenize(text.lower())
    filtered_tokens=[lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token)>2]
    return filtered_tokens

In [33]:
df['cleaned_text']=df['ABSTRACT'].apply(clean_text)

In [34]:
dictionary=corpora.Dictionary(df['cleaned_text'])

In [35]:
dictionary.filter_extremes(no_below=20,no_above=0.5)

In [37]:
corpus=[dictionary.doc2bow(text) for text in df['cleaned_text']]

In [38]:
lda_model=models.LdaModel(corpus,num_topics=4,id2word=dictionary,passes=15)

In [40]:
for idx,topic in lda_model.print_topics(-1):
    print(f'Topic:{idx}\nWords:{topic}\n')

Topic:0
Words:0.010*"space" + 0.010*"result" + 0.010*"graph" + 0.009*"group" + 0.008*"mathbb" + 0.008*"function" + 0.008*"prove" + 0.007*"show" + 0.007*"equation" + 0.007*"number"

Topic:1
Words:0.008*"model" + 0.008*"energy" + 0.008*"system" + 0.007*"field" + 0.007*"phase" + 0.007*"state" + 0.006*"two" + 0.005*"time" + 0.005*"quantum" + 0.005*"effect"

Topic:2
Words:0.015*"algorithm" + 0.015*"model" + 0.015*"method" + 0.014*"problem" + 0.009*"function" + 0.008*"time" + 0.008*"distribution" + 0.007*"data" + 0.006*"show" + 0.006*"based"

Topic:3
Words:0.016*"network" + 0.012*"data" + 0.012*"model" + 0.010*"learning" + 0.008*"based" + 0.007*"approach" + 0.007*"system" + 0.007*"method" + 0.006*"paper" + 0.006*"using"



In [42]:
df['topic'] = df['cleaned_text'].apply(lambda x: lda_model.get_document_topics(dictionary.doc2bow(x))[0][0])

In [43]:
df.to_csv('output with topics.csv',index=False)